## Research commodity options volume

In [1]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr

# importlib.reload(build_db)

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



## IF YOU WANT TO SEE WARNINGS, COMMENT THIS OUT

In [2]:
import warnings
warnings.filterwarnings("ignore")

### important global variables

In [3]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'


In [4]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    return fig

def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
#             x0=ld_beg[i],
            x0=ld_beg,
            y0=0,
#             x1=ld_end[i],
            x1=ld_end,
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

#### get all contracts in the options database

In [5]:
pga = db_info.get_db_info()
print(f"futtab max date: {pga.get_sql(f'select max(settle_date) from {futtab}')}")
print(f"opttab max date: {pga.get_sql(f'select max(settle_date) from {opttab}')}")


  sec_db
futtab max date:         max
0  20200515
opttab max date:         max
0  20200515


### analyse option volumes by day, to see largest volumes

In [6]:
vol_oi_dates_old = pga.get_sql(f"select distinct settle_date from {opttab} order by settle_date desc").settle_date.unique()
vol_oi_dates_new = np.append(vol_oi_dates_old[1:],[-1])
df_vol_oi_dates = pd.DataFrame({'settle_date':vol_oi_dates_old,'new_settle_date':vol_oi_dates_new})


In [7]:
def acs_adjust_vol_oi_date(df_with_wrong_vol_oi_date):
    df_new = df_with_wrong_vol_oi_date.merge(df_vol_oi_dates,how='inner',on='settle_date')
    df_new.settle_date = df_new.new_settle_date
    df_new = df_new[df_new.settle_date>-1]
    return df_new


In [8]:
sql_cl_opt_vol_by_date = f"""
select strike,pc,volume,open_interest  from sec_schema.options_table 
where settle_date = 20200302 and symbol = 'CLM20'
order by strike,pc 
"""
df_sql_cl_opt_vol_by_date = pga.get_sql(sql_cl_opt_vol_by_date)
# df_sql_cl_opt_vol_by_date[(df_sql_cl_opt_vol_by_date.pc=='C') & (df_sql_cl_opt_vol_by_date.volsum>0)]
df_sql_cl_opt_vol_by_date[(df_sql_cl_opt_vol_by_date.pc=='C') ].head(40)


,strike,pc,volume,open_interest
0,20.5,C,0,0
2,21.0,C,0,1
3,22.0,C,1,1
5,22.5,C,1,1
7,23.0,C,1,0
9,23.5,C,0,0
11,24.0,C,0,0
12,24.5,C,0,0
14,25.0,C,0,2
16,25.5,C,0,1


In [9]:

sql_cl_opt_vol_by_date = f"""
select settle_date,sum(volume) volsum, sum(open_interest) oi_sum from sec_schema.options_table 
where settle_date >= 20200220 and symbol like 'CL%%'
group by settle_date
order by settle_date desc
"""
df_volsum = pga.get_sql(sql_cl_opt_vol_by_date)


In [10]:
acs_adjust_vol_oi_date(df_volsum)

,settle_date,volsum,oi_sum,new_settle_date
0,20200514,53503,1173637,20200514
1,20200513,70126,1751055,20200513
2,20200512,81424,1751554,20200512
3,20200511,71324,1749692,20200511
4,20200508,45323,1746273,20200508
...,...,...,...,...
56,20200225,159727,1914747,20200225
57,20200224,153643,1879440,20200224
58,20200221,120900,1852108,20200221
59,20200220,111082,1823665,20200220


In [11]:
b = df_volsum.settle_date.values
print(b)
b = b[1:]
print(np.append(b,[-1]))

[20200515 20200514 20200513 20200512 20200511 20200508 20200507 20200506
 20200505 20200504 20200501 20200430 20200429 20200428 20200427 20200424
 20200423 20200422 20200421 20200420 20200417 20200416 20200415 20200414
 20200413 20200409 20200408 20200407 20200406 20200403 20200402 20200401
 20200331 20200330 20200327 20200326 20200325 20200324 20200323 20200320
 20200319 20200318 20200317 20200316 20200313 20200312 20200311 20200310
 20200309 20200306 20200305 20200304 20200303 20200302 20200228 20200227
 20200226 20200225 20200224 20200221 20200220]
[20200514 20200513 20200512 20200511 20200508 20200507 20200506 20200505
 20200504 20200501 20200430 20200429 20200428 20200427 20200424 20200423
 20200422 20200421 20200420 20200417 20200416 20200415 20200414 20200413
 20200409 20200408 20200407 20200406 20200403 20200402 20200401 20200331
 20200330 20200327 20200326 20200325 20200324 20200323 20200320 20200319
 20200318 20200317 20200316 20200313 20200312 20200311 20200310 20200309
 202

In [12]:
sql_cl_opts_vol_by_put_call = f"""
with
f1 as (
    select settle_date,pc,symbol,sum(volume) vol_sum_by_pc from sec_schema.options_table
    where settle_date>=20200101 and symbol like 'CL%%' and volume>0
    group by settle_date, pc,symbol
    order by settle_date, pc,symbol
),
f2 as
(
    select settle_date,symbol,sum(vol_sum_by_pc) vol_sum_by_symbol
    from f1
    group by settle_date,symbol
),
f3 as
(
    select f1.*, f2.vol_sum_by_symbol from f1
    join f2 on f1.settle_date  = f2.settle_date and f1.symbol = f2.symbol
    order by f2.vol_sum_by_symbol desc,f1.settle_date,f1.pc
)
select * from f3
"""
df_cl_opts_vol_by_put_call = pga.get_sql(sql_cl_opts_vol_by_put_call)


In [13]:
df_cl_opts_vol_by_put_call_C = df_cl_opts_vol_by_put_call[df_cl_opts_vol_by_put_call.pc=='C']
df_cl_opts_vol_by_put_call_C = df_cl_opts_vol_by_put_call_C.rename(columns={'vol_sum_by_pc':'vol_sum_C'})
df_cl_opts_vol_by_put_call_C = df_cl_opts_vol_by_put_call_C[['settle_date','symbol','vol_sum_C','vol_sum_by_symbol']]
df_cl_opts_vol_by_put_call_C.index = list(range(len(df_cl_opts_vol_by_put_call_C)))
df_cl_opts_vol_by_put_call_P = df_cl_opts_vol_by_put_call[df_cl_opts_vol_by_put_call.pc=='P']
df_cl_opts_vol_by_put_call_P = df_cl_opts_vol_by_put_call_P.rename(columns={'vol_sum_by_pc':'vol_sum_P'})
df_cl_opts_vol_by_put_call_P = df_cl_opts_vol_by_put_call_P[['settle_date','symbol','vol_sum_P']]
df_cl_opts_vol_by_put_call_P.index = list(range(len(df_cl_opts_vol_by_put_call_P)))
df_cl_opts_vol = df_cl_opts_vol_by_put_call_C.merge(df_cl_opts_vol_by_put_call_P,how='inner',
                                                 on=['settle_date','symbol'])
df_cl_opts_vol['pc_vol_diff'] = df_cl_opts_vol.vol_sum_C - df_cl_opts_vol.vol_sum_P
df_cl_opts_vol = acs_adjust_vol_oi_date(df_cl_opts_vol)
display.display(df_cl_opts_vol[['settle_date','symbol','vol_sum_C','vol_sum_P','pc_vol_diff','vol_sum_by_symbol']].head(20))


2020-05-18 14:51:11,544 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


,settle_date,symbol,vol_sum_C,vol_sum_P,pc_vol_diff,vol_sum_by_symbol
0,20200108,CLG20,113253,96419,16834,209672.0
1,20200108,CLH20,74577,45366,29211,119943.0
2,20200108,CLJ20,15235,13857,1378,29092.0
3,20200108,CLM20,16210,12757,3453,28967.0
4,20200108,CLK20,4202,5671,-1469,9873.0
5,20200108,CLZ20,4762,3488,1274,8250.0
6,20200108,CLN20,5697,2378,3319,8075.0
7,20200108,CLU20,2539,2295,244,4834.0
8,20200108,CLZ21,2251,200,2051,2451.0
9,20200108,CLM21,1450,601,849,2051.0


In [14]:
df_cl_opts_vol_sum_by_date = df_cl_opts_vol[['settle_date','vol_sum_C','vol_sum_P','vol_sum_by_symbol']].groupby('settle_date').sum()
df_cl_opts_vol_sum_by_date = df_cl_opts_vol_sum_by_date.sort_values('settle_date',ascending=False)
df_cl_opts_vol_sum_by_date

,vol_sum_C,vol_sum_P,vol_sum_by_symbol
settle_date,,,
20200514,24154,28999,53153.0
20200513,36134,33377,69511.0
20200512,32014,48756,80770.0
20200511,26742,44182,70924.0
20200508,20789,24430,45219.0
...,...,...,...
20200107,88035,67930,155965.0
20200106,134921,76670,211591.0
20200103,165801,112425,278226.0


In [15]:
sql_opt_expir = f"""
with
f1 as
(
    select symbol,concat(substring(symbol,1,2),substring(symbol,4,2),substring(symbol,3,1)) as sym,settle_date
    from sec_schema.options_table
    where settle_date>=20160101 and symbol like 'CL%%' and volume>0
)
select * from f1
"""
df_sql_opt_expir = pga.get_sql(sql_opt_expir)


In [16]:
df_sql_opt_expir_last_day = df_sql_opt_expir[['settle_date','symbol','sym']].groupby(['symbol','sym'],as_index=False).max()
df_sql_opt_expir_last_day = df_sql_opt_expir_last_day[df_sql_opt_expir_last_day.sym<='CL20G']
df_sql_opt_expir_last_day


,symbol,sym,settle_date
0,CLF17,CL17F,20161215
1,CLF18,CL18F,20171214
2,CLF19,CL19F,20181214
3,CLF20,CL20F,20191216
6,CLG16,CL16G,20160114
7,CLG17,CL17G,20170117
8,CLG18,CL18G,20180117
9,CLG19,CL19G,20190116
10,CLG20,CL20G,20200115
12,CLH16,CL16H,20160217


In [17]:
sql_cl_opts_vol_by_date = f"""
with
f1 as (
    select settle_date,volume from sec_schema.options_table
    where settle_date>=20160101 and symbol like 'CL%%' and volume>0
),
f2 as
(
    select settle_date,sum(volume) vol_sum
    from f1
    group by settle_date
    order by settle_date
)
select * from f2
"""
df_cl_opts_vol_date = pga.get_sql(sql_cl_opts_vol_by_date)


In [18]:
fig = go.Figure()

# Add scatter trace for line
fig.add_trace(go.Scatter(
    x=["2015-02-01", "2015-02-02", "2015-02-03", "2015-02-04", "2015-02-05",
       "2015-02-06", "2015-02-07", "2015-02-08", "2015-02-09", "2015-02-10",
       "2015-02-11", "2015-02-12", "2015-02-13", "2015-02-14", "2015-02-15",
       "2015-02-16", "2015-02-17", "2015-02-18", "2015-02-19", "2015-02-20",
       "2015-02-21", "2015-02-22", "2015-02-23", "2015-02-24", "2015-02-25",
       "2015-02-26", "2015-02-27", "2015-02-28"],
    y=[-14, -17, -8, -4, -7, -10, -12, -14, -12, -7, -11, -7, -18, -14, -14,
       -16, -13, -7, -8, -14, -8, -3, -9, -9, -4, -13, -9, -6],
    mode="lines",
    name="temperature"
))

# Add shape regions
fig.update_layout(
    shapes=[
        # 1st highlight during Feb 4 - Feb 6
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0="2015-02-04",
            y0=0,
            x1="2015-02-06",
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        ),
        # 2nd highlight during Feb 20 - Feb 23
        dict(
            type="rect",
            xref="x",
            yref="paper",
            x0="2015-02-20",
            y0=0,
            x1="2015-02-22",
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
    ]
)

fig.show()

In [19]:
df_cl_opts_vol_date2 = df_cl_opts_vol_date.copy()
df_cl_opts_vol_date2['is_ld'] = df_cl_opts_vol_date2.settle_date.apply(
    lambda d:1 if d in df_sql_opt_expir_last_day.settle_date.unique() else 0)
df_cl_opts_vol_date2['yyyy'] = df_cl_opts_vol_date2.settle_date.apply(lambda d:int(str(d)[0:4]))   
for y in np.sort(df_cl_opts_vol_date2.yyyy.unique()):
    df_yyyy = df_cl_opts_vol_date2[df_cl_opts_vol_date2.yyyy==y]
    fig = plotly_plot(
            df_in=df_yyyy[['settle_date','vol_sum']],
            x_column='settle_date',
            plot_title=f'Option Volume {y} (Expiration in Orange)',
            y_left_label='Total Options Volume Per Day'
        )
    
    df_yyyy['is_ld_end'] = df_yyyy.is_ld.shift(1)
    df_yyyy['is_ld_beg'] = df_yyyy.is_ld.shift(-1)
    ld_beg = df_yyyy[df_yyyy.is_ld_beg==1].settle_date.values
    ld_end = df_yyyy[df_yyyy.is_ld_end==1].settle_date.values

    ld_shapes = []
    for i in range(min(len(ld_beg),len(ld_end))):
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    iplot(fig)


In [20]:
sql_cl_opts_oi_by_put_call = f"""
with
f1 as (
    select settle_date,pc,symbol,sum(open_interest) oi_sum_by_pc from sec_schema.options_table
    where settle_date>=20200225 and symbol like 'CL%%' and open_interest>0
    group by settle_date, pc,symbol
    order by settle_date, pc,symbol
),
f2 as
(
    select settle_date,symbol,sum(oi_sum_by_pc) oi_sum_by_symbol
    from f1
    group by settle_date,symbol
),
f3 as
(
    select f1.*, f2.oi_sum_by_symbol from f1
    join f2 on f1.settle_date  = f2.settle_date and f1.symbol = f2.symbol
    order by f2.oi_sum_by_symbol desc,f1.settle_date,f1.pc
)
select * from f3
"""
df_cl_opts_oi_by_put_call = pga.get_sql(sql_cl_opts_oi_by_put_call)




In [21]:
df_cl_opts_oi_by_put_call_C = df_cl_opts_oi_by_put_call[df_cl_opts_oi_by_put_call.pc=='C']
df_cl_opts_oi_by_put_call_C = df_cl_opts_oi_by_put_call_C.rename(columns={'oi_sum_by_pc':'oi_sum_C'})
df_cl_opts_oi_by_put_call_C = df_cl_opts_oi_by_put_call_C[['settle_date','symbol','oi_sum_C','oi_sum_by_symbol']]
df_cl_opts_oi_by_put_call_P = df_cl_opts_oi_by_put_call[df_cl_opts_oi_by_put_call.pc=='P']
df_cl_opts_oi_by_put_call_P = df_cl_opts_oi_by_put_call_P.rename(columns={'oi_sum_by_pc':'oi_sum_P'})
df_cl_opts_oi_by_put_call_P = df_cl_opts_oi_by_put_call_P[['settle_date','symbol','oi_sum_P']]
df_cl_opts_oi = df_cl_opts_oi_by_put_call_C.merge(df_cl_opts_oi_by_put_call_P,how='inner',
                                                 on=['settle_date','symbol'])
df_cl_opts_oi['pc_oi_diff'] = df_cl_opts_oi.oi_sum_C - df_cl_opts_oi.oi_sum_P
display.display(df_cl_opts_oi[['settle_date','symbol','oi_sum_C','oi_sum_P','pc_oi_diff','oi_sum_by_symbol']].head(20))


,settle_date,symbol,oi_sum_C,oi_sum_P,pc_oi_diff,oi_sum_by_symbol
0,20200422,CLM20,507484,493255,14229,1000739.0
1,20200424,CLM20,515585,482233,33352,997818.0
2,20200423,CLM20,512926,484781,28145,997707.0
3,20200421,CLM20,489018,507744,-18726,996762.0
4,20200420,CLM20,479220,483166,-3946,962386.0
5,20200417,CLM20,474992,465001,9991,939993.0
6,20200416,CLM20,469415,451540,17875,920955.0
7,20200415,CLM20,465652,437458,28194,903110.0
8,20200414,CLM20,456207,423570,32637,879777.0
9,20200413,CLM20,454172,418769,35403,872941.0


In [22]:
df_cl_opts_oi_sum_by_date = df_cl_opts_oi[['settle_date','oi_sum_by_symbol']].groupby('settle_date').sum()
df_cl_opts_oi_sum_by_date = df_cl_opts_oi_sum_by_date.sort_values('oi_sum_by_symbol',ascending=False)
df_cl_opts_oi_sum_by_date

,oi_sum_by_symbol
settle_date,
20200415,2946283.0
20200416,2907882.0
20200414,2892753.0
20200413,2848551.0
20200409,2807532.0
20200408,2768401.0
20200407,2740856.0
20200406,2717254.0
20200424,2711201.0


In [23]:
# df_cl_opts_oi.to_csv('temp_folder/LO_open_interest_by_day.csv',index=False)

In [24]:
len(df_cl_opts_oi)

1124

In [25]:
sql = f"""
select settle_date,open_interest from {futtab} f
where f.settle_date>=20180101 and f.contract_num=CN and substring(symbol,1,2)='CL'
"""
sql1 = sql.replace('CN','1')
df1 = pga.get_sql(sql1)
sql2 = sql.replace('CN','2')
df2 = pga.get_sql(sql2)


In [26]:
df1_oi = df1.rename(columns={'open_interest':'m1_oi'})
df2_oi = df2.rename(columns={'open_interest':'m2_oi'})
dfb = df1_oi.merge(df2_oi,on='settle_date',how='inner')
dfb['oi_diff'] = dfb.m1_oi - dfb.m2_oi

In [27]:
iplot(plotly_plot(df_in=dfb[['settle_date','oi_diff']],x_column='settle_date',plot_title='Open Interest Month1 - Open Interest Month2'))

In [28]:
dfb2 = dfb[dfb.settle_date>=20200120]
fig = plotly_plot(df_in=dfb2[['settle_date','oi_diff']],x_column='settle_date',
                 plot_title='Open Interest Month1 - Open Interest Month2 (Last 4 rolls)')
shade_dates = [(20200121,20200212),(20200220,20200311),(20200320,20200413),(20200421,20200427)]
fig = plotly_shaded_rectangles(shade_dates,fig)
iplot(fig)

## END